

## Contributors

1. xx
2. xx
3. xx
4. xx
5. xx

In [5]:
# Installation of nltk
# In Jupyter, the console commands can be executed by the ‘!’ sign before the command within the cell
!pip install nltk
!pip install openpyxl




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\lashreev\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\lashreev\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Develop an application to preprocess a document collection for effective Boolean query retrieval. Implement a function that takes a dictionary of documents (where document IDs serve as keys and text content as values) and constructs an inverted index from the pre-processed text. Compute and display the number of tokens at various stages of preprocessing:
A.    Before preprocessing
B.    After stopword removal
C.    After normalization
D.    After stemming/lemmatization. 

In [6]:
import os
from docx import Document
import PyPDF2
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define text extraction functions for various formats
def read_docx(file_path):
    doc = Document(file_path)
    return " ".join([para.text for para in doc.paragraphs])

def read_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text() if page.extract_text() else ""
    return text

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_xlsx(file_path):
    excel_data = pd.ExcelFile(file_path)
    all_sheets_text = []
    for sheet_name in excel_data.sheet_names:
        df = excel_data.parse(sheet_name)
        all_sheets_text.append(" ".join(df.astype(str).stack()))
    return " ".join(all_sheets_text)

# Load and process all files from a dataset directory
def load_documents(dataset_path):
    document_collection = {}
    for root, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.docx'):
                document_collection[file] = read_docx(file_path)
            elif file.endswith('.pdf'):
                document_collection[file] = read_pdf(file_path)
            elif file.endswith('.txt'):
                document_collection[file] = read_txt(file_path)
            elif file.endswith('.xlsx'):
                document_collection[file] = read_xlsx(file_path)
    return document_collection

# Preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    token_count_before = len(tokens)

    # Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    token_count_stopwords_removed = len(tokens)

    # Normalization (lowercasing, removing punctuation)
    tokens = [word.lower() for word in tokens if word.isalnum()]
    token_count_normalized = len(tokens)

    # Stemming/Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    token_count_lemmatized = len(tokens)

    return tokens, token_count_before, token_count_stopwords_removed, token_count_normalized, token_count_lemmatized

# Build inverted index
def build_inverted_index(documents):
    inverted_index = defaultdict(set)
    for doc_id, text in documents.items():
        tokens, count_before, count_stopwords_removed, count_normalized, count_lemmatized = preprocess_text(text)
        for token in tokens:
            inverted_index[token].add(doc_id)
        
        # Display token counts for the document
        print(f"Document: {doc_id}")
        print(f"Tokens before preprocessing: {count_before}")
        print(f"Tokens after stopword removal: {count_stopwords_removed}")
        print(f"Tokens after normalization: {count_normalized}")
        print(f"Tokens after lemmatization: {count_lemmatized}\n")
    return inverted_index

# Main execution
if __name__ == "__main__":
    dataset_path = "C:\\Users\\lashreev\\Downloads\\Dataset"  
    documents = load_documents(dataset_path)
    inverted_index = build_inverted_index(documents)

    # Display a sample of the inverted index
    print("Sample of Inverted Index:")
    for term, doc_ids in list(inverted_index.items())[:10]:
        print(f"{term}: {doc_ids}")


Document: Nature_1.docx
Tokens before preprocessing: 167
Tokens after stopword removal: 110
Tokens after normalization: 84
Tokens after lemmatization: 84

Document: Nature_10.pdf
Tokens before preprocessing: 155
Tokens after stopword removal: 99
Tokens after normalization: 79
Tokens after lemmatization: 79

Document: Nature_2.pdf
Tokens before preprocessing: 197
Tokens after stopword removal: 114
Tokens after normalization: 85
Tokens after lemmatization: 85

Document: Nature_3.txt
Tokens before preprocessing: 144
Tokens after stopword removal: 96
Tokens after normalization: 63
Tokens after lemmatization: 63

Document: Nature_4.xlsx
Tokens before preprocessing: 119
Tokens after stopword removal: 74
Tokens after normalization: 61
Tokens after lemmatization: 61

Document: Nature_5.docx
Tokens before preprocessing: 85
Tokens after stopword removal: 57
Tokens after normalization: 46
Tokens after lemmatization: 46

Document: Nature_6.pdf
Tokens before preprocessing: 171
Tokens after stopword

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lashreev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lashreev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lashreev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2.     Implement dictionary compression techniques to estimate space savings and compare their efficiency. Techniques to be implemented include:
·       Without blocking
·       With blocking
·       Blocking combined with front coding. (for both dictionary and posting lists)
Analyze the space utilization of the compressed dictionary for each method and present the results.

In [7]:
import os
from docx import Document
import PyPDF2
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import sys
import struct

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define text extraction functions for various formats
def read_docx(file_path):
    doc = Document(file_path)
    return " ".join([para.text for para in doc.paragraphs])

def read_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text() if page.extract_text() else ""
    return text

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_xlsx(file_path):
    excel_data = pd.ExcelFile(file_path)
    all_sheets_text = []
    for sheet_name in excel_data.sheet_names:
        df = excel_data.parse(sheet_name)
        all_sheets_text.append(" ".join(df.astype(str).stack()))
    return " ".join(all_sheets_text)

# Load and process all files from a dataset directory
def load_documents(dataset_path):
    document_collection = {}
    for root, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.docx'):
                document_collection[file] = read_docx(file_path)
            elif file.endswith('.pdf'):
                document_collection[file] = read_pdf(file_path)
            elif file.endswith('.txt'):
                document_collection[file] = read_txt(file_path)
            elif file.endswith('.xlsx'):
                document_collection[file] = read_xlsx(file_path)
    return document_collection

# Preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    token_count_before = len(tokens)

    # Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    token_count_stopwords_removed = len(tokens)

    # Normalization (lowercasing, removing punctuation)
    tokens = [word.lower() for word in tokens if word.isalnum()]
    token_count_normalized = len(tokens)

    # Stemming/Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    token_count_lemmatized = len(tokens)

    return tokens, token_count_before, token_count_stopwords_removed, token_count_normalized, token_count_lemmatized

# Build inverted index
def build_inverted_index(documents):
    inverted_index = defaultdict(set)
    for doc_id, text in documents.items():
        tokens, count_before, count_stopwords_removed, count_normalized, count_lemmatized = preprocess_text(text)
        for token in tokens:
            inverted_index[token].add(doc_id)
        
        # Display token counts for the document
        print(f"Document: {doc_id}")
        print(f"Tokens before preprocessing: {count_before}")
        print(f"Tokens after stopword removal: {count_stopwords_removed}")
        print(f"Tokens after normalization: {count_normalized}")
        print(f"Tokens after lemmatization: {count_lemmatized}\n")
    return inverted_index

# Compression without blocking
def compress_without_blocking(dictionary):
    compressed_dict = {}  # Store term and offset
    current_offset = 0
    for term in sorted(dictionary):
        compressed_dict[term] = current_offset
        current_offset += len(term) + 1  # +1 for null terminator
    return compressed_dict

# Compression with blocking
def compress_with_blocking(dictionary, block_size):
    compressed_dict = {}
    current_offset = 0
    block = []
    block_start = None

    for idx, term in enumerate(sorted(dictionary)):
        if idx % block_size == 0:
            block_start = term
            block = []
            compressed_dict[block_start] = current_offset
        block.append(term)
        current_offset += len(term) + 1

    return compressed_dict

# Compression with blocking and front coding
def compress_with_front_coding(dictionary, block_size):
    compressed_dict = {}
    current_offset = 0
    block = []
    block_start = None

    for idx, term in enumerate(sorted(dictionary)):
        if idx % block_size == 0:
            block_start = term
            block = []
            compressed_dict[block_start] = current_offset
            prev_term = term
            current_offset += len(term) + 1
        else:
            prefix_length = 0
            while (prefix_length < len(prev_term) and prefix_length < len(term) and prev_term[prefix_length] == term[prefix_length]):
                prefix_length += 1
            compressed_dict[f"{prefix_length}|{term[prefix_length:]}"] = current_offset
            current_offset += len(term) - prefix_length + 1
            prev_term = term

    return compressed_dict

# Calculate dictionary size
def calculate_size(dictionary):
    return sum(sys.getsizeof(key) + sys.getsizeof(value) for key, value in dictionary.items())

# Main execution
if __name__ == "__main__":
    dataset_path = "C:\\Users\\lashreev\\Downloads\\Dataset" 
    documents = load_documents(dataset_path)
    inverted_index = build_inverted_index(documents)

    # Compress the dictionary
    dictionary = {term: idx for idx, term in enumerate(sorted(inverted_index.keys()))}

    compressed_without_blocking = compress_without_blocking(dictionary)
    compressed_with_blocking = compress_with_blocking(dictionary, block_size=8)
    compressed_with_front_coding = compress_with_front_coding(dictionary, block_size=8)

    # Analyze space savings
    original_size = calculate_size(dictionary)
    size_without_blocking = calculate_size(compressed_without_blocking)
    size_with_blocking = calculate_size(compressed_with_blocking)
    size_with_front_coding = calculate_size(compressed_with_front_coding)

    print("Space Utilization Analysis:")
    print(f"Original Dictionary Size: {original_size} bytes")
    print(f"Compressed Without Blocking: {size_without_blocking} bytes")
    print(f"Compressed With Blocking: {size_with_blocking} bytes")
    print(f"Compressed With Front Coding: {size_with_front_coding} bytes")

    # Display a sample of the compressed dictionary
    print("\nSample of Compressed Dictionary:")
    for term, offset in list(compressed_with_front_coding.items())[:10]:
        print(f"{term}: {offset}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lashreev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lashreev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lashreev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Document: Nature_1.docx
Tokens before preprocessing: 167
Tokens after stopword removal: 110
Tokens after normalization: 84
Tokens after lemmatization: 84

Document: Nature_10.pdf
Tokens before preprocessing: 155
Tokens after stopword removal: 99
Tokens after normalization: 79
Tokens after lemmatization: 79

Document: Nature_2.pdf
Tokens before preprocessing: 197
Tokens after stopword removal: 114
Tokens after normalization: 85
Tokens after lemmatization: 85

Document: Nature_3.txt
Tokens before preprocessing: 144
Tokens after stopword removal: 96
Tokens after normalization: 63
Tokens after lemmatization: 63

Document: Nature_4.xlsx
Tokens before preprocessing: 119
Tokens after stopword removal: 74
Tokens after normalization: 61
Tokens after lemmatization: 61

Document: Nature_5.docx
Tokens before preprocessing: 85
Tokens after stopword removal: 57
Tokens after normalization: 46
Tokens after lemmatization: 46

Document: Nature_6.pdf
Tokens before preprocessing: 171
Tokens after stopword